In [ ]:
import re
from collections import namedtuple

# Token 定義
Token = namedtuple('Token', ['type', 'value'])

def tokenize(code):
    # Simplified SKIP pattern - focus on whitespace first
    # Will handle comments manually or with separate logic
    token_spec = [
        ('OP',       r'==|!=|<=|>=|[+\-*/<>]'),
        ('EQ',       r'='),
        ('LPAREN',   r'\('),
        ('RPAREN',   r'\)'),
        ('LBRACE',   r'\{'),
        ('RBRACE',   r'\}'),
        ('COMMA',    r','),
        ('SEMI',     r';'),
        ('NUMBER',   r'\d+'),
        ('ID',       r'[A-Za-z_][A-Za-z0-9_]*'), # Match ID after keywords/types if they were separate patterns
        ('WHITESPACE', r'\s+'), # Keep whitespace pattern separate for now
    ]

    regex = '|'.join(f'(?P<{name}>{pattern})' for name, pattern in token_spec)
    tokens = []
    pos = 0
    while pos < len(code):
        # Check for multi-line comments first
        if code[pos:].startswith('/*'):
            end_comment = code.find('*/', pos + 2)
            if end_comment == -1:
                raise Exception("Unterminated multi-line comment")
            pos = end_comment + 2
            continue # Skip to the next iteration

        # Check for single-line comments
        if code[pos:].startswith('//'):
            end_comment = code.find('\n', pos + 2)
            if end_comment == -1:
                pos = len(code) # Consume till end of file if no newline
            else:
                pos = end_comment + 1
            continue # Skip to the next iteration


        match = re.match(regex, code[pos:], re.DOTALL)
        if match:
            kind = match.lastgroup
            value = match.group(0)

            if kind == 'WHITESPACE':
                pos += len(value)
                continue # Skip whitespace

            # Handle keywords and types after matching ID
            if kind == 'ID':
                if value in ('int', 'char'):
                    kind = 'TYPE'
                elif value in ('if', 'else', 'while', 'for', 'return'):
                    kind = 'KEYWORD'
            tokens.append(Token(kind, value))
            pos += len(value)
        else:
            # If no pattern matches, there's an illegal character
            raise Exception(f"Illegal character at position {pos}: {code[pos]}")

    tokens.append(Token('EOF', '')) # Add EOF token

    return tokens

# AST 節點 (Assuming these are correctly defined)
class ASTNode:
    pass

class Program(ASTNode):
    def __init__(self, functions):
        self.functions = functions

class Function(ASTNode):
    def __init__(self, ret_type, name, params, body):
        self.ret_type = ret_type
        self.name = name
        self.params = params
        self.body = body

class VarDecl(ASTNode):
    def __init__(self, var_type, name, init=None):
        self.var_type = var_type
        self.name = name
        self.init = init

class Block(ASTNode):
    def __init__(self, stmts):
        self.stmts = stmts

class IfStmt(ASTNode):
    def __init__(self, cond, then_body, else_body=None):
        self.cond = cond
        self.then_body = then_body
        self.else_body = else_body

class WhileStmt(ASTNode):
    def __init__(self, cond, body):
        self.cond = cond
        self.body = body

class ForStmt(ASTNode):
    def __init__(self, init, cond, step, body):
        self.init = init
        self.cond = cond
        self.step = step
        self.body = body

class ReturnStmt(ASTNode):
    def __init__(self, expr):
        self.expr = expr

class Assignment(ASTNode):
    def __init__(self, target, expr):
        self.target = target
        self.expr = expr

class BinaryOp(ASTNode):
    def __init__(self, op, left, right):
        self.op = op
        self.left = left
        self.right = right

class FuncCall(ASTNode):
    def __init__(self, name, args):
        self.name = name
        self.args = args

class Var(ASTNode):
    def __init__(self, name):
        self.name = name

class Literal(ASTNode):
    def __init__(self, value):
        self.value = value

# Parser
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0
        # Simple set of known function names for parsing "f a" syntax
        # In a real compiler, this would come from a symbol table after declarations.
        self.known_functions = { 'main'}

    def peek(self):
        return self.tokens[self.pos] if self.pos < len(self.tokens) else Token('EOF', '')

    def peek_next(self):
        return self.tokens[self.pos + 1] if self.pos + 1 < len(self.tokens) else Token('EOF', '')


    def consume(self, expected_type=None, expected_value=None):
        token = self.peek()
        if token.type == 'EOF' and expected_type != 'EOF':
            raise Exception("Unexpected EOF")

        if expected_type and token.type != expected_type:
            raise Exception(f"Expected {expected_type} but got {token.type} with value '{token.value}'")
        if expected_value and token.value != expected_value:
            raise Exception(f"Expected {expected_value} but got '{token.value}'")
        self.pos += 1
        return token

    def consume_type(self):
        tok = self.peek()
        if tok and tok.type == 'TYPE':
            return self.consume().value
        else:
            raise Exception(f"Expected type but got {tok.type if tok else 'EOF'}")

    def parse_program(self):
        functions = []
        while self.peek().type != 'EOF':
            if self.peek().type != 'TYPE':
                 raise Exception(f"Expected function definition (starting with a type), but got {self.peek().type}")
            functions.append(self.parse_function())

        self.consume('EOF')

        return Program(functions)

    def parse_function(self):
        ret_type = self.consume_type()
        name = self.consume('ID').value
        # Add function name to known functions for parsing "f a" inside
        self.known_functions.add(name)
        self.consume('LPAREN')
        params = []
        if self.peek().type != 'RPAREN':
            params.append(self.parse_param())
            while self.peek() and self.peek().type == 'COMMA':
                self.consume('COMMA')
                params.append(self.parse_param())
        self.consume('RPAREN')
        body = self.parse_block()
        return Function(ret_type, name, params, body)

    def parse_param(self):
        t = self.consume_type()
        n = self.consume('ID').value
        return VarDecl(t, n)


    def parse_block(self):
        stmts = []
        self.consume('LBRACE')
        while self.peek().type != 'RBRACE':
            if self.peek().type == 'EOF':
                 raise Exception("Unexpected EOF within a block - missing closing brace '}'?")
            stmts.append(self.parse_stmt())
        self.consume('RBRACE')
        return Block(stmts)

    def parse_stmt(self):
        tok = self.peek()
        if tok.type == 'EOF':
            raise Exception("Unexpected EOF in statement")

        if tok.type == 'TYPE':
            return self.parse_vardecl()
        elif tok.type == 'KEYWORD':
             if tok.value == 'if':
                  return self.parse_if()
             elif tok.value == 'while':
                  return self.parse_while()
             elif tok.value == 'for':
                  return self.parse_for()
             elif tok.value == 'return':
                  self.consume('KEYWORD', 'return')
                  expr = self.parse_expr()
                  self.consume('SEMI')
                  return ReturnStmt(expr)
             else:
                  raise Exception(f"Unknown keyword: {tok.value}")
        else:
            return self.parse_expr_stmt()


    def parse_vardecl(self):
        t = self.consume_type()
        n = self.consume('ID').value
        init = None
        if self.peek() and self.peek().type == 'EQ':
            self.consume('EQ')
            init = self.parse_expr()
        self.consume('SEMI')
        return VarDecl(t, n, init)

    def parse_expr_stmt(self):
        expr = self.parse_expr()
        self.consume('SEMI')
        if isinstance(expr, BinaryOp) and expr.op == '=':
            if not isinstance(expr.left, Var):
                 raise Exception("Invalid assignment target: left side must be a variable")
            return Assignment(expr.left, expr.right)
        # Allow standalone function calls as statements
        if isinstance(expr, FuncCall):
             return expr
        # Other standalone expressions are not typically statements in C-like languages
        # For now, we'll just return them.
        return expr


    def parse_if(self):
        self.consume('KEYWORD', 'if')
        self.consume('LPAREN')
        cond = self.parse_expr()
        self.consume('RPAREN')
        then_body = self.parse_block()
        else_body = None
        if self.peek() and self.peek().type == 'KEYWORD' and self.peek().value == 'else':
            self.consume('KEYWORD', 'else')
            else_body = self.parse_block()
        return IfStmt(cond, then_body, else_body)

    def parse_while(self):
        self.consume('KEYWORD', 'while')
        self.consume('LPAREN')
        cond = self.parse_expr()
        self.consume('RPAREN')
        body = self.parse_block()
        return WhileStmt(cond, body)

    def parse_for(self):
        self.consume('KEYWORD', 'for')
        self.consume('LPAREN')

        init = None
        # Check if the initialization is a variable declaration
        if self.peek().type == 'TYPE':
             init = self.parse_vardecl()
        elif self.peek().type != 'SEMI':
             # Otherwise, it's an expression statement (like assignment)
             init = self.parse_expr()
             self.consume('SEMI')
             # If the expression was an assignment, wrap it in an Assignment AST node
             if isinstance(init, BinaryOp) and init.op == '=':
                 if not isinstance(init.left, Var):
                      raise Exception("Invalid for loop init assignment target")
                 init = Assignment(init.left, init.right)
        else:
             # Empty initialization part
             self.consume('SEMI')

        cond = None
        if self.peek().type != 'SEMI':
            cond = self.parse_expr()
        self.consume('SEMI')

        step = None
        if self.peek().type != 'RPAREN':
            step = self.parse_expr()
            # If the expression was an assignment, wrap it in an Assignment AST node
            if isinstance(step, BinaryOp) and step.op == '=':
                 if not isinstance(step.left, Var):
                     raise Exception("Invalid for loop step assignment target")
                 step = Assignment(step.left, step.right)


        self.consume('RPAREN')
        body = self.parse_block()

        return ForStmt(init, cond, step, body)


    PRECEDENCE = {
        '=': 1,
        '<': 2, '>': 2, '<=': 2, '>=': 2, '==': 2, '!=': 2,
        '+': 3, '-': 3,
        '*': 4, '/': 4, # Implicit multiplication will have the same precedence as explicit
        'implicit_call': 5, # Higher precedence for implicit function calls
        'implicit_mul': 4 # Implicit multiplication has the same precedence as explicit
    }

    # Implemented using precedence climbing with implicit multiplication and function calls
    def parse_expr(self, min_precedence=0):
        left = self.parse_primary()

        while True:
            tok = self.peek()

            if tok.type == 'EOF' or tok.type in ('SEMI', 'COMMA', 'RPAREN'):
                 break

            # Check for explicit operators
            if tok.type in ('OP', 'EQ'):
                op = tok.value
                current_precedence = self.PRECEDENCE.get(op, -1)

                if current_precedence < min_precedence:
                     break

                self.consume() # Consume explicit operator
                next_min_precedence = current_precedence + (0 if op == '=' else 1)
                right = self.parse_expr(next_min_precedence)
                left = BinaryOp(op, left, right)
                continue # Continue the loop with the new 'left' node

            # Check for implicit operations (function call or multiplication)
            # An implicit operation starts if the next token can be the start of a primary
            if tok.type in ('ID', 'NUMBER', 'LPAREN'):
                # Determine the type of implicit operation and its precedence
                op = None
                current_precedence = -1

                # If 'left' is a variable and its name is a known function, it could be an implicit function call
                if isinstance(left, Var) and left.name in self.known_functions:
                    op = 'implicit_call'
                    current_precedence = self.PRECEDENCE['implicit_call']
                else:
                    # Otherwise, assume implicit multiplication
                    op = 'implicit_mul'
                    current_precedence = self.PRECEDENCE['implicit_mul']

                if current_precedence < min_precedence:
                     break

                # For implicit operations, parse the right-hand side (the argument or the second operand)
                # The minimum precedence for the right-hand side depends on the operation type.
                # Implicit calls are right-associative or have higher precedence, implicit multiplication is left-associative.
                next_min_precedence = current_precedence # Implicit calls are often treated as right-associative or just high precedence
                if op == 'implicit_mul':
                    next_min_precedence = current_precedence + 1 # Implicit multiplication is left-associative

                right = self.parse_expr(next_min_precedence)

                if op == 'implicit_call':
                    # If it was an implicit function call, 'right' is the argument
                    left = FuncCall(left.name, [right])
                elif op == 'implicit_mul':
                    # If it was an implicit multiplication
                    left = BinaryOp('*', left, right)

                continue # Continue the loop with the new 'left' node


            # If the token is not an operator or a primary start, break the loop
            break


        return left


    def parse_primary(self):
        tok = self.peek()
        if tok.type == 'EOF':
             raise Exception("Unexpected EOF in primary expression")

        if tok.type == 'NUMBER':
            self.consume()
            return Literal(int(tok.value))
        elif tok.type == 'ID':
            name = self.consume('ID').value
            # Check for explicit function call syntax: ID followed by LPAREN
            if self.peek() and self.peek().type == 'LPAREN':
                self.consume('LPAREN')
                args = []
                if self.peek().type != 'RPAREN':
                    args.append(self.parse_expr())
                    while self.peek() and self.peek().type == 'COMMA':
                        self.consume('COMMA')
                        args.append(self.parse_expr())
                self.consume('RPAREN')
                return FuncCall(name, args)
            else:
                # It's just a variable reference: ID not followed by LPAREN
                return Var(name)

        elif tok.type == 'LPAREN':
            self.consume('LPAREN')
            node = self.parse_expr()
            self.consume('RPAREN')
            return node
        else:
            raise Exception(f"Unexpected token {tok} in primary expression")

# TAC Generator
class TACGenerator:
    def __init__(self):
        self.temp_count = 0
        self.label_count = 0
        self.code = []
        self.variables = {} # Not used in this simple version, but would be for symbol table

    def new_temp(self):
        self.temp_count += 1
        return f't{self.temp_count}'

    def new_label(self):
        self.label_count += 1
        return f'L{self.label_count}'

    def gen(self, node):
        method = 'gen_' + node.__class__.__name__
        if not hasattr(self, method):
             # Handle statement nodes or expressions that can be statements
             if isinstance(node, (BinaryOp, Var, Literal)):
                 # If it's an expression node used as a statement, just generate its value (and discard)
                 self.gen_expr(node)
                 return
             else:
                 raise NotImplementedError(f'Method {method} not implemented for {node.__class__.__name__}')
        # If the method exists, call it (for Program, Function, Block, If, While, For, Return)
        return getattr(self, method)(node)

    def gen_expr(self, node):
         # Helper to generate code for expression nodes
         method = 'gen_' + node.__class__.__name__
         if not hasattr(self, method):
              raise NotImplementedError(f'Expression generator {method} not implemented for {node.__class__.__name__}')
         return getattr(self, method)(node)


    def gen_Program(self, node):
        for f in node.functions:
            self.gen(f)
        return self.code

    def gen_Function(self, node):
        self.code.append(f'func {node.name}:')
        for i, param in enumerate(node.params):
             self.code.append(f'  # param {param.name} ({param.var_type})')

        self.gen(node.body)
        self.code.append(f'endfunc')
        self.temp_count = 0 # Reset temp count for each function

    def gen_Block(self, node):
        for stmt in node.stmts:
            self.gen(stmt)

    def gen_VarDecl(self, node):
        # Variable declaration might have an initial assignment
        if node.init:
            val = self.gen_expr(node.init)
            self.code.append(f'  {node.name} = {val}')
        else:
             # Default initialization (e.g., to 0 for int)
             self.code.append(f'  {node.name} = 0')


    def gen_Assignment(self, node):
        val = self.gen_expr(node.expr)
        if not isinstance(node.target, Var):
            # This check should ideally be done during semantic analysis,
            # but we'll keep it here for basic validation.
            raise Exception(f"Assignment target must be a variable, but got {node.target.__class__.__name__}")
        self.code.append(f'  {node.target.name} = {val}')

    def gen_ReturnStmt(self, node):
        val = self.gen_expr(node.expr)
        self.code.append(f'  return {val}')

    def gen_IfStmt(self, node):
        cond = self.gen_expr(node.cond)
        label_else = self.new_label()
        label_end = self.new_label()

        self.code.append(f'  if_false {cond} goto {label_else}')
        self.gen(node.then_body)
        self.code.append(f'  goto {label_end}')
        self.code.append(f'{label_else}:')
        if node.else_body:
            self.gen(node.else_body)
        self.code.append(f'{label_end}:')

    def gen_WhileStmt(self, node):
        label_start = self.new_label()
        label_end = self.new_label()

        self.code.append(f'{label_start}:')
        cond = self.gen_expr(node.cond)
        self.code.append(f'  if_false {cond} goto {label_end}')
        self.gen(node.body)
        self.code.append(f'  goto {label_start}')
        self.code.append(f'{label_end}:')

    def gen_ForStmt(self, node):
        label_start = self.new_label()
        label_end = self.new_label()
        label_step = self.new_label() # Label for the step part

        if node.init:
             self.gen(node.init)

        self.code.append(f'{label_start}:')

        if node.cond:
            cond_val = self.gen_expr(node.cond)
            self.code.append(f'  if_false {cond_val} goto {label_end}')

        # Generate body code
        self.gen(node.body)

        # Jump to step, then back to condition
        self.code.append(f'  goto {label_step}')

        self.code.append(f'{label_step}:')
        if node.step:
             self.gen(node.step)

        self.code.append(f'  goto {label_start}') # Loop back to condition check

        self.code.append(f'{label_end}:')


    def gen_BinaryOp(self, node):
        left_val = self.gen_expr(node.left)
        right_val = self.gen_expr(node.right)
        temp = self.new_temp()
        self.code.append(f'  {temp} = {left_val} {node.op} {right_val}')
        return temp

    def gen_FuncCall(self, node):
        arg_temps = []
        for arg_node in node.args:
             arg_temps.append(self.gen_expr(arg_node))

        result_temp = self.new_temp()
        # In TAC, function calls are often represented as `result = call function_name, arg1, arg2, ...`
        arg_list_str = ', '.join(arg_temps)
        self.code.append(f'  {result_temp} = call {node.name}, {arg_list_str}')

        return result_temp

    def gen_Var(self, node):
        return node.name

    def gen_Literal(self, node):
        return str(node.value)

# Test case with implicit multiplication and nested calls
code = """
/* This is a multi-line comment
   spanning multiple lines */
int f(int x) {
  int y = x + 1; // Single-line comment
  return y;
}
int g(int x) {
  int y = x 3; // Single-line comment
  return y;
}

int main() {
  int a = 3;
  int b = 3 (3+2)(2+1)5+ f f (3 a 3 g a +1+a f a a+2g(2+ a) 2)a; // Test f f a * a should be f(f(a)) * a
  int c = a b + 2; // Test a b + 2 should be a * b + 2
  int d = 2 f(5); // Test 2 f(5) should be 2 * f(5)
  if (b > 4) {
    b = b - 1;
  } else {
    b = b + 1;
  }
  for (int i = 0; i < 10; i = i + 1) {
    b = b + i;
  }
  return b;
}
"""

print("Step 1 & 2: Tokenizing the code and printing tokens...")
tokens = tokenize(code)
print("Full token list generated by tokenizer:")
for token in tokens:
    print(token)
print("-" * 20)

print("Step 3: Parsing tokens to build AST...")
parser = Parser(tokens)
ast = parser.parse_program()
print("Parsing successful. AST generated.")
# Optional: print the AST for debugging
# def print_ast(node, level=0):
#     indent = "  " * level
#     if isinstance(node, list):
#         print(f"{indent}[")
#         for item in node:
#             print_ast(item, level + 1)
#         print(f"{indent}]")
#     elif isinstance(node, ASTNode):
#         print(f"{indent}{node.__class__.__name__}(")
#         for attr, value in node.__dict__.items():
#             if isinstance(value, ASTNode) or isinstance(value, list):
#                 print(f"{indent}  {attr}=")
#                 print_ast(value, level + 1)
#             else:
#                  print(f"{indent}  {attr}={repr(value)},\n")
#         print(f"{indent})")
#     else:
#         print(f"{indent}{repr(node)}")
# print_ast(ast)
print("-" * 20)

print("Step 4 & 5: Generating TAC from AST and printing TAC code...")
tacgen = TACGenerator()
tac_code = tacgen.gen(ast)

print("Generated TAC Code:")
for line in tac_code:
    print(line)